# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from glob import glob

from pyopia.classify import Classify
import exampledata
import pyopia.io
from pyopia.pipeline import Pipeline
from pyopia.instrument.silcam import SilCamLoad, ImagePrep
import pyopia.process

In [ ]:
filename = exampledata.get_example_silc_image()

files = glob('raw_data/*.silc')

In [ ]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

datafile_hdf = 'proc/test'
model_path = exampledata.get_example_model()
threshold = 0.85

steps = {'classifier': Classify(model_path=model_path),
         'load': SilCamLoad(),
         'imageprep': ImagePrep(),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps, initial_steps=['classifier'])

In [ ]:
for filename in files:
    stats = processing_pipeline.run(filename)

In [ ]:
# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

In [ ]:
import pyopia.statistics
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))
print('Number of raw images: ', pyopia.statistics.count_images_in_stats(stats))